In [1]:
import json
import logging
import xml.etree.ElementTree as ET
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
filepath = os.path.normpath(os.getcwd() + '/..' + '/data/processed/data.json')
filepath

'/home/hazot/code/trec-clinical-trials-2023/data/processed/data.json'

In [3]:
df = pd.read_json(filepath)

In [4]:
print(df.shape)
print(df.columns)

(451538, 17)
Index(['nct_id', 'link_text', 'url', 'id_info', 'brief_title', 'sponsors',
       'brief_summary', 'detailed_description', 'primary_purpose',
       'intervention', 'eligibility', 'gender', 'minimum_age', 'maximum_age',
       'healthy_volunteers', 'keyword', 'condition_browse'],
      dtype='object')


In [16]:
print("Sum of column:", np.sum(list(df['gender'].value_counts())))
print('-----------')
print(df['gender'].value_counts())

Sum of column: 450667
gender
All       387617
Female     43441
Male       19609
Name: count, dtype: int64


In [17]:
print("Sum of column:", np.sum(list(df['minimum_age'].value_counts())))
print('-----------')
print(df['minimum_age'].value_counts())

Sum of column: 450667
minimum_age
18 Years     279575
N/A           30526
20 Years      17938
40 Years       9334
21 Years       9238
              ...  
84 Days           1
118 Years         1
73 Hours          1
167 Days          1
23 Days           1
Name: count, Length: 318, dtype: int64


In [18]:
print("Sum of column:", np.sum(list(df['maximum_age'].value_counts())))
print('-----------')
print(df['maximum_age'].value_counts())

Sum of column: 450667
-----------
maximum_age
N/A           214375
65 Years       29033
80 Years       23198
75 Years       22838
70 Years       18586
               ...  
54 Days            1
263 Months         1
93 Days            1
26 Hours           1
37 Days            1
Name: count, Length: 473, dtype: int64


In [19]:
print("Sum of column:", np.sum(list(df['healthy_volunteers'].value_counts())))
print('-----------')
print(df['healthy_volunteers'].value_counts())

Sum of column: 440292
-----------
healthy_volunteers
No                            322154
Accepts Healthy Volunteers    118138
Name: count, dtype: int64


### More complicated cases

In [20]:
df['intervention'][0]

[{'intervention_type': 'Biological',
  'intervention_name': 'V930',
  'description': 'V930 Over a 94 week duration, patients will receive a series of 5 injections (2.5 mg/injection), one every other week. Within 2 minutes of each injection of V930, each patient will be given an EP-IM injection consisting of two 60 msec pulses.',
  'arm_group_label': '1'},
 {'intervention_type': 'Biological',
  'intervention_name': 'V932',
  'description': 'V932 Over a 94 week duration, patients will receive a series of 5 injections, 6 patients will initially received intramuscular V932 vaccinations at a low dose (0.5x109 vg/injection),and following a safety assessment, up to an additional 35 patients will be treated with the high dose V932 (0.5x1011 vg/injection).',
  'arm_group_label': '2'}]

In [24]:
df['eligibility'][0].keys()

dict_keys(['criteria', 'gender', 'minimum_age', 'maximum_age', 'healthy_volunteers'])

In [22]:
df['keyword'][:10]

0                  Cancers expressing HER-2 and/or CEA
1                                                 None
2    [dexmedetomidine, midazolam, remifentanil, Ele...
3                                                 None
4    [adenocarcinoma of the colon, stage I colon ca...
5                                                 None
6                                                 None
7                                                 None
8    [ADHD, ADD, Attention Deficit Hyperactivity Di...
9     [wound healing, phototoxicity, Healthy Subjects]
Name: keyword, dtype: object

In [ ]:
df['condition_browse'][:10]

### Less complicated

In [ ]:
df['primary_purpose'].value_counts()

In [ ]:
df['nct_id'].value_counts()

In [ ]:
count_url = 0
for i in range(len(df['url'])):
    count_url += 1 if df['url'][i] else 0
print(count_url)

In [ ]:
len(df['id_info'][2].keys())

In [ ]:
max_keys = 0
max_string_keys = ''
for i in range(len(df['id_info'])):
    if max_keys >= len(df['id_info'][i].keys()):
        break
    max_keys = len(df['id_info'][i].keys())
    max_string_keys = df['id_info'][i].keys()
print(max_keys)
print(max_string_keys)

min_keys = 100
min_string_keys = ''
for i in range(len(df['id_info'])):
    if min_keys <= len(df['id_info'][i].keys()):
        break
    min_keys = len(df['id_info'][i].keys())
    min_string_keys = df['id_info'][i].keys()
print(min_keys)
print(min_string_keys)

In [ ]:
df['id_info'][0].keys()

In [ ]:
count_nct_id = 0
for i in range(len(df['id_info'])):
    count_nct_id += 1 if df['id_info'][i].get('nct_id', 0) else 0
print(count_nct_id)

In [ ]:
df['brief_title']

In [ ]:
df['brief_title'].isna().sum()

In [ ]:
df['brief_summary'].isna().sum()

In [ ]:
df['brief_summary'].apply(type).value_counts()

In [ ]:
df['brief_summary'].value_counts()[:10]

In [ ]:
bad_indexes = df.loc[pd.isna(df["brief_summary"]), :].index
print(bad_indexes)
print('len(bad_indexes):', len(bad_indexes))

### Check word (tokens) frequency

1. Word frequency in all strings

In [ ]:
df['brief_summary'][:10000].str.split(expand=True).stack().value_counts()[:30]

In [ ]:
df['brief_summary']

2. Token frequency in all strings

In [ ]:
tokenizer = None